In [1]:
!pip install kaggle

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,185 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126675 files and dire

In [4]:
import os, json, math, random, zipfile
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
from kaggle.api.kaggle_api_extended import KaggleApi
import torch

In [8]:
os.environ['KAGGLE_USERNAME'] = 'starsin'
os.environ['KAGGLE_KEY'] = 'ff559a02afd29d67f19bc4001d5a69f7'

api = KaggleApi()
api.authenticate()


# 2. 대회 데이터 다운로드
competition_name = "ai05-level1-project"
api.competition_download_files(competition_name, path="./data")

In [9]:
zip_path = "./data/ai05-level1-project.zip"
extract_dir = "./data/ai05-level1-project"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

- 데이터 예시
  - K-003351-016232-020014_0_2_0_2_75_000_200.png = 이미지
  - K-003351-016232-020014 = 어노테이션
    - K-003351-016232-020014_0_2_0_2_75_000_200.json = 어노테이션 정보들이 여기에 들어있음



In [10]:
# =========================
# 설정
# =========================
base_path = "./data/ai05-level1-project"
train_img_dir = os.path.join(base_path, "train_images")
train_ann_dir = os.path.join(base_path, "train_annotations")

NUM_SAMPLES = 10      # 무작위로 볼 이미지 개수
RANDOM_SEED = 42      # 재현성을 위해 고정
IOU_NMS_THR = 0.6     # 중복 박스 제거 임계값 (0.5~0.7 권장)

In [11]:
def iou(box1, box2):
    x1,y1,w1,h1 = box1
    x2,y2,w2,h2 = box2
    xa, ya = max(x1,x2), max(y1,y2)
    xb, yb = min(x1+w1, x2+w2), min(y1+h1, y2+h2)
    inter = max(0, xb-xa) * max(0, yb-ya)
    a1 = w1*h1; a2 = w2*h2
    union = a1 + a2 - inter + 1e-9
    return inter/union

def nms_by_iou(anns, iou_thr=0.5):
    kept = []
    for a in anns:
        ok = True
        for b in kept:
            if iou(a.get("bbox", [0,0,0,0]), b.get("bbox", [0,0,0,0])) > iou_thr:
                ok = False
                break
        if ok:
            kept.append(a)
    return kept

In [12]:
def collect_coco_for_image(ann_dir, image_file):
    merged = {"images": [], "annotations": [], "categories": {}}
    image_obj = None
    ann_id = 1

    for dp,_,files in os.walk(ann_dir):
        for f in files:
            if not f.lower().endswith(".json"):
                continue
            jp = os.path.join(dp,f)
            with open(jp,"r") as fh:
                j = json.load(fh)

            images = j.get("images", [])
            anns   = j.get("annotations", [])
            cats   = j.get("categories", [])

            for c in cats:
                merged["categories"][c["id"]] = c

            matched_ids = []
            for im in images:
                fn = os.path.basename(im.get("file_name", "") or im.get("imgfile",""))
                if fn == image_file:
                    matched_ids.append(im["id"])
                    if image_obj is None:
                        image_obj = {
                            "id": im["id"],
                            "file_name": image_file,
                            "width": im.get("width"),
                            "height": im.get("height")
                        }

            for a in anns:
                if matched_ids and a.get("image_id") in matched_ids:
                    new_a = dict(a)
                    new_a["id"] = ann_id; ann_id += 1
                    merged["annotations"].append(new_a)

    merged["images"] = [image_obj] if image_obj else []
    merged["categories"] = list(merged["categories"].values())
    return merged

In [13]:
def count_jsons_for_image(ann_dir, image_file):
    count = 0
    target_json_name = image_file.replace(".png", ".json")
    found_paths = []
    for dp,_,files in os.walk(ann_dir):
        for f in files:
            if f.lower().endswith(".json") and target_json_name in f:
                count += 1
                found_paths.append(os.path.join(dp, f))
    return count, found_paths

In [14]:
def draw_merged(merged, image_path, subtitle=""):
    img = np.array(Image.open(image_path).convert("RGB"))
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(img); ax.axis("off")

    cat_map = {c["id"]: c.get("name", str(c["id"])) for c in merged["categories"]}

    anns = merged["annotations"]
    if anns:
        anns = nms_by_iou(anns, iou_thr=IOU_NMS_THR)

    for a in anns:
        bbox = a.get("bbox")
        if not bbox or len(bbox) != 4:
            continue
        x,y,w,h = bbox
        rect = patches.Rectangle((x,y), w,h, fill=False, linewidth=2)
        ax.add_patch(rect)
        label = cat_map.get(a.get("category_id"), str(a.get("category_id")))
        ax.text(x, max(0,y-5), label, fontsize=9, backgroundcolor="white")

    ttl = os.path.basename(image_path)
    if subtitle:
        ttl += f"\n{subtitle}"
    plt.title(ttl)
    plt.show()

In [15]:
# 이미지 후보 수집
img_files = sorted([f for f in os.listdir(train_img_dir) if f.lower().endswith((".png",".jpg",".jpeg"))])
random.Random(RANDOM_SEED).shuffle(img_files)
sampled = img_files[:min(NUM_SAMPLES, len(img_files))]

print(f"✅ 총 이미지 {len(img_files)}장 중 {len(sampled)}장 랜덤 선택")

for image_file in sampled:
    img_path = os.path.join(train_img_dir, image_file)

    # prefix_json 폴더 찾기
    prefix = image_file.split('_')[0]  # 'K-xxxx-xxxx-...'
    cand = [d for d in os.listdir(train_ann_dir) if d.startswith(prefix) and d.endswith("_json")]
    if not cand:
        print(f"❌ 어노테이션 폴더 없음: {image_file}")
        continue
    ann_dir = os.path.join(train_ann_dir, cand[0])

    # 디버그: 해당 이미지명 JSON 존재 개수
    cnt, paths = count_jsons_for_image(ann_dir, image_file)
    print(f"\n🖼️ {image_file}")
    print(f"📂 ann_dir = {os.path.basename(ann_dir)}")
    print(f"🔎 매칭되는 JSON 파일 수 = {cnt}")
    if cnt == 0:
        print("   ⚠️ 정확 파일명 매칭 JSON 없음. 그래도 병합 시도.")
    else:
        for p in paths[:3]:
            print("   ✅", p)
        if len(paths) > 3:
            print(f"   … (외 {len(paths)-3}개)")

    # 병합 & 시각화
    merged = collect_coco_for_image(ann_dir, image_file)
    subtitle = f"Merged anns: {len(merged['annotations'])}  |  Matched JSONs by name: {cnt}"
    draw_merged(merged, img_path, subtitle=subtitle)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
# 1️⃣ train_annotations 전체를 하나의 COCO로 병합
merged = {"images": [], "annotations": [], "categories": []}
cat_map = {}
imgid_remap = {}  # (prefix_file_name, orig_image_id) -> new_image_id
ann_id = 1
new_img_id = 1

for folder in os.listdir(train_ann_dir):
    if not folder.endswith("_json"):
        continue
    ann_dir = os.path.join(train_ann_dir, folder)
    for dp, _, files in os.walk(ann_dir):
        for f in files:
            if not f.endswith(".json"):
                continue
            with open(os.path.join(dp, f), "r") as fh:
                j = json.load(fh)
            images = j.get("images", [])
            anns   = j.get("annotations", [])
            cats   = j.get("categories", [])
            # 카테고리 병합
            for c in cats:
                if c["id"] not in cat_map:
                    cat_map[c["id"]] = c
            # 이미지/어노테이션 리맵
            for im in images:
                file_name = im.get("file_name") or im.get("imgfile")
                file_name = os.path.basename(file_name)
                key = (file_name, im["id"])
                if key not in imgid_remap:
                    imgid_remap[key] = new_img_id
                    merged["images"].append({
                        "id": new_img_id,
                        "file_name": file_name,
                        "width": im.get("width", 1024),
                        "height": im.get("height", 1024)
                    })
                    new_img_id += 1
            for a in anns:
                file_name = None
                # image_id 매핑을 위해 원본 image_id가 어떤 파일인지 찾아야 함
                # 간단히 해당 json의 images 테이블에서 동일 id를 찾아 file_name을 얻는다
                for im in images:
                    if im["id"] == a["image_id"]:
                        file_name = os.path.basename(im.get("file_name") or im.get("imgfile"))
                        mapped = imgid_remap[(file_name, im["id"])]
                        new_a = dict(a)
                        new_a["id"] = ann_id; ann_id += 1
                        new_a["image_id"] = mapped
                        merged["annotations"].append(new_a)
                        break

merged["categories"] = list(cat_map.values())

os.makedirs("./merged_coco", exist_ok=True)
with open("./merged_coco/train_all.json", "w") as f:
    json.dump(merged, f, indent=2)
print("✅ COCO 병합 완료:", len(merged["images"]), "images,", len(merged["annotations"]), "anns,", len(merged["categories"]), "cats")

✅ COCO 병합 완료: 1489 images, 4526 anns, 73 cats


In [17]:
# torchvision Detection용 Dataset (수정판: 0-based 라벨 매핑)
class CocoLikeDetection(torch.utils.data.Dataset):
    def __init__(self, img_dir, coco_json, transforms=None):
        with open(coco_json, "r") as f:
            coco = json.load(f)
        self.img_dir = img_dir
        self.transforms = transforms

        # images / img_ids
        self.images  = {im["id"]: im for im in coco["images"]}
        self.img_ids = list(self.images.keys())

        # image_id -> annotations
        self.anns_by_img = {im_id: [] for im_id in self.img_ids}
        for a in coco["annotations"]:
            self.anns_by_img[a["image_id"]].append(a)

        # 카테고리 집합 및 0-based 매핑
        self.cat_ids = sorted({c["id"] for c in coco["categories"]})
        self.cat_id_to_idx = {cid: i for i, cid in enumerate(self.cat_ids)}   # ✅ 0..num_classes-1
        self.idx_to_catid  = {i: cid for cid, i in self.cat_id_to_idx.items()} # 역매핑(추론/CSV용)

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id  = self.img_ids[idx]
        im_meta = self.images[img_id]
        img_path = os.path.join(self.img_dir, im_meta["file_name"])
        img = Image.open(img_path).convert("RGB")

        boxes, labels, areas, iscrowd = [], [], [], []
        for a in self.anns_by_img.get(img_id, []):
            bbox = a.get("bbox")
            if not bbox or len(bbox) != 4:
                continue
            x, y, w, h = bbox
            boxes.append([x, y, x + w, y + h])

            # ✅ 반드시 0-based로 변환
            cid = a["category_id"]
            labels.append(self.cat_id_to_idx.get(cid, 0))

            areas.append(a.get("area", w * h))
            iscrowd.append(a.get("iscrowd", 0))

        target = {
            "boxes":   torch.as_tensor(boxes, dtype=torch.float32)   if boxes else torch.zeros((0,4), dtype=torch.float32),
            "labels":  torch.as_tensor(labels, dtype=torch.int64)    if labels else torch.zeros((0,),  dtype=torch.int64),
            "image_id": torch.tensor([img_id]),
            "area":    torch.as_tensor(areas, dtype=torch.float32)   if areas else torch.zeros((0,),   dtype=torch.float32),
            "iscrowd": torch.as_tensor(iscrowd, dtype=torch.int64)   if iscrowd else torch.zeros((0,), dtype=torch.int64),
        }

        if self.transforms:
            img = self.transforms(img)
        return img, target

In [18]:
import os, json, random, torch
import torchvision.transforms as T
from PIL import Image

# 병합한 COCO 어노테이션 경로 (앞에서 merge한 json)
train_json = "./merged_coco/train_all.json"

# ===============================
# 2️⃣ Transform 정의
# ===============================
def get_transform():
    """
    이미지를 Tensor로 변환하는 기본 transform
    (필요시 RandomHorizontalFlip 등 augmentation 추가 가능)
    """
    return T.Compose([T.ToTensor()])

# ===============================
# 3️⃣ Collate 함수 (dataloader용)
# ===============================
def collate_fn(batch):
    """
    detection 모델용 collate 함수
    """
    return tuple(zip(*batch))

In [24]:
# 2️⃣ train/val split
full_ds = CocoLikeDetection(train_img_dir, train_json, transforms=get_transform())
indices = list(range(len(full_ds)))
random.Random(0).shuffle(indices)
split = int(0.9*len(indices))
train_idx, val_idx = indices[:split], indices[split:]

train_ds = torch.utils.data.Subset(full_ds, train_idx)
val_ds   = torch.utils.data.Subset(full_ds, val_idx)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=2, collate_fn=collate_fn)

num_classes = len(full_ds.cat_ids)   # 실제 객체 클래스 수
print("✅ train/val:", len(train_ds), len(val_ds), "classes:", num_classes)

✅ train/val: 1340 149 classes: 73


In [25]:
# 3️⃣ RetinaNet (one-stage detector)
import torchvision
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torchvision의 RetinaNet은 num_classes=카테고리 개수(배경 제외)로 지정
model = torchvision.models.detection.retinanet_resnet50_fpn_v2(weights=None, num_classes=num_classes)
model = model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=2.5e-4, weight_decay=1e-4)
lr_sched  = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=12)

In [26]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0):
        """
        patience: 개선 없는 epoch 허용 횟수
        delta: 최소 개선 폭 (예: 0.001보다 작으면 개선 아님)
        """
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            print(f"⚠️ EarlyStopping 카운터 {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [27]:
# early stopper 인스턴스 생성
early_stopper = EarlyStopping(patience=3, delta=0.005)

# 4️⃣ Train loop (수정 안정판)
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    for imgs, targets in train_loader:
        imgs = [im.to(device) for im in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # --- 방어 코드: 라벨 범위 체크 (RetinaNet은 0-based 필요) ---
        for t in targets:
            if t["labels"].numel() > 0:
                assert int(t["labels"].min()) >= 0, f"❌ 잘못된 라벨(min={int(t['labels'].min())})"
                assert int(t["labels"].max()) < num_classes, \
                    f"❌ 잘못된 라벨(max={int(t['labels'].max())}) >= num_classes({num_classes})"

        # --- 손실 계산 ---
        loss_dict = model(imgs, targets)
        loss = sum(loss_dict.values())

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    lr_sched.step()

    # =======================
    # Validation (간단 박스 수 체크)
    # =======================
    model.eval()
    val_loss = 0.0
    total_boxes = 0
    avg_score = 0

    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = [im.to(device) for im in imgs]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            model.train()   # ✅ 추가 (loss 계산용)
            # 🔹 validation 손실 계산
            loss_dict = model(imgs, targets)
            loss = sum(loss_dict.values())
            model.eval()    # ✅ 추가 (다시 eval 복귀)
            val_loss += loss.item()

            # 🔹 inference 결과 확인 (optional)
            outs = model(imgs)
            total_boxes += sum(o["boxes"].shape[0] for o in outs)
            avg_score += np.mean([
                float(o["scores"].mean()) if len(o["scores"]) > 0 else 0 for o in outs
            ])

    val_loss /= len(val_loader)
    avg_score /= len(val_loader)


    print(f"[{epoch+1}/{EPOCHS}] "
      f"train_loss={total_loss/len(train_loader):.4f} | "
      f"val_loss={val_loss:.4f} | "
      f"val_boxes={total_boxes} | avg_score={avg_score:.3f}")

        # ✅ EarlyStopping 체크
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print(f"⏹ Early stopping triggered at epoch {epoch+1}")
        break
# =======================
# 모델 저장
# =======================
save_path = "./data/retinanet_pills.pth"
torch.save(model.state_dict(), save_path)
print(f"💾 모델 저장 완료: {save_path}")

[1/20] train_loss=0.8836 | val_loss=0.6300 | val_boxes=21641 | avg_score=0.134
[2/20] train_loss=0.4930 | val_loss=0.4223 | val_boxes=17832 | avg_score=0.164
[3/20] train_loss=0.3721 | val_loss=0.3858 | val_boxes=14361 | avg_score=0.144
[4/20] train_loss=0.3139 | val_loss=0.3205 | val_boxes=12157 | avg_score=0.140
[5/20] train_loss=0.2779 | val_loss=0.2950 | val_boxes=11708 | avg_score=0.134
[6/20] train_loss=0.2474 | val_loss=0.2820 | val_boxes=10723 | avg_score=0.140
[7/20] train_loss=0.2214 | val_loss=0.2555 | val_boxes=11529 | avg_score=0.144
[8/20] train_loss=0.1987 | val_loss=0.2395 | val_boxes=9534 | avg_score=0.142
[9/20] train_loss=0.1817 | val_loss=0.2243 | val_boxes=8610 | avg_score=0.138
[10/20] train_loss=0.1648 | val_loss=0.2202 | val_boxes=7338 | avg_score=0.138
⚠️ EarlyStopping 카운터 1/3
[11/20] train_loss=0.1534 | val_loss=0.2115 | val_boxes=7229 | avg_score=0.143
[12/20] train_loss=0.1451 | val_loss=0.2075 | val_boxes=6961 | avg_score=0.143
⚠️ EarlyStopping 카운터 1/3
[13/

In [ ]:
# 5️⃣ test_images에서 추론 후 CSV로 저장 (최종 안정판)
import pandas as pd
from pathlib import Path

test_dir = os.path.join(base_path, "test_images")
score_thr = 0.25
csv_out  = "./stage1_retinanet_predictions.csv"

rows = []
annotation_id = 1
model.eval()

# label 인덱스 ↔ 실제 카테고리 id 매핑
idx_to_catid = {i+1: cid for i, cid in enumerate(full_ds.cat_ids)}

file_list = [fn for fn in os.listdir(test_dir) if fn.lower().endswith((".png", ".jpg", ".jpeg"))]
file_list.sort()

for fn in file_list:
    img_path = os.path.join(test_dir, fn)
    img_name = os.path.splitext(fn)[0]  # ✅ 확장자 제거 (예: "1.png" → "1")
    img = Image.open(img_path).convert("RGB")
    x = T.ToTensor()(img).to(device)

    with torch.no_grad():
        out = model([x])[0]

    boxes  = out["boxes"].detach().cpu().numpy()
    labels = out["labels"].detach().cpu().numpy()
    scores = out["scores"].detach().cpu().numpy()

    for (x1, y1, x2, y2), lab, sc in zip(boxes, labels, scores):
        if sc < score_thr:
            continue
        if lab == 0:  # ✅ category_id 0(배경)은 제외
            continue

        rows.append({
            "annotation_id": annotation_id,
            "image_id": img_name,  # ✅ 확장자 제거된 이름 사용
            "category_id": int(idx_to_catid.get(int(lab), int(lab))),
            "bbox_x": round(float(x1), 2),
            "bbox_y": round(float(y1), 2),
            "bbox_w": round(float(x2 - x1), 2),
            "bbox_h": round(float(y2 - y1), 2),
            "score": round(float(sc), 3)
        })
        annotation_id += 1

# ✅ file_name 제거, image_id에 확장자 없는 이름만 포함
df = pd.DataFrame(rows, columns=[
    "annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])
df.to_csv(csv_out, index=False, encoding="utf-8-sig")
print(f"✅ CSV 저장 완료: {csv_out} (rows={len(df)})")

✅ CSV 저장: ./stage1_retinanet_predictions.csv  (rows=4687)


In [ ]:
# 벨리데이션 set을 따로 만들고, 학습을 별도로 하게된다.

In [38]:
import torch
import torchvision.ops as ops

def nms_per_class_keep_top1(
    boxes, labels, scores,
    score_thr=0.25,
    iou_thr=0.5,
    keep_by="area",   # "area" 또는 "score"
    verbose=False
):
    """
    입력:
      boxes [N,4] (xyxy), labels [N], scores [N]  — CPU float32 권장
    동작:
      1) score_thr 이하, label==0(배경) 제거
      2) 클래스별 NMS (IoU)
      3) 남은 박스 중 '클래스당 1개'만 유지 (면적 or 점수 기준)
    반환:
      boxes_kept, labels_kept, scores_kept (모두 CPU float32 / long)
    """
    # --- 0) 안전 캐스팅 & 정리 ---
    if not isinstance(boxes, torch.Tensor):  boxes  = torch.as_tensor(boxes)
    if not isinstance(labels, torch.Tensor): labels = torch.as_tensor(labels)
    if not isinstance(scores, torch.Tensor): scores = torch.as_tensor(scores)

    boxes  = boxes.detach().to("cpu", dtype=torch.float32).contiguous()
    labels = labels.detach().to("cpu", dtype=torch.int64).contiguous()
    scores = scores.detach().to("cpu", dtype=torch.float32).contiguous()

    # --- 1) 기본 필터 ---
    keep_mask = (scores >= float(score_thr)) & (labels != 0)
    boxes, labels, scores = boxes[keep_mask], labels[keep_mask], scores[keep_mask]
    if boxes.numel() == 0:
        if verbose: print("NMS: no boxes after score/bg filter")
        return boxes, labels, scores

    kept_b, kept_l, kept_s = [], [], []

    # --- 2) 클래스별 NMS ---
    uniq = labels.unique().tolist()
    for lab in uniq:
        m = (labels == lab)
        b = boxes[m]
        s = scores[m]
        if b.numel() == 0:
            continue

        # IoU 기반 중복 제거: 점수는 s 사용
        keep_idx = ops.nms(b, s, float(iou_thr))
        b2, s2 = b[keep_idx], s[keep_idx]

        # --- 3) 클래스당 1개만 남기기 (면적 or 점수 기준)
        if b2.shape[0] > 1:
            if keep_by == "score":
                chosen = torch.argmax(s2)
            else:  # keep_by == "area"
                areas = (b2[:, 2] - b2[:, 0]).clamp(min=0) * (b2[:, 3] - b2[:, 1]).clamp(min=0)
                chosen = torch.argmax(areas)
            b2 = b2[chosen:chosen+1]
            s2 = s2[chosen:chosen+1]

        kept_b.append(b2)
        kept_l.append(torch.full((b2.shape[0],), lab, dtype=torch.int64))
        kept_s.append(s2)

    if len(kept_b) == 0:
        if verbose: print("NMS: nothing kept per-class")
        return boxes.new_zeros((0,4)), labels.new_zeros((0,), dtype=torch.int64), scores.new_zeros((0,))

    boxes_kept  = torch.cat(kept_b, dim=0)
    labels_kept = torch.cat(kept_l, dim=0)
    scores_kept = torch.cat(kept_s, dim=0)

    if verbose:
        print(f"NMS kept: {boxes_kept.shape[0]} boxes (classes={len(uniq)})")

    return boxes_kept, labels_kept, scores_kept

In [43]:
# ==== Per-Drug (category_id) NMS → 클래스별 '가장 큰 박스 1개'만 남기기 ====
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T
import torchvision.ops as ops
from PIL import Image

# --- 설정 ---
test_dir    = os.path.join(base_path, "test_images")
csv_out     = "./stage1_retinanet_predictions_one_per_drug_nms.csv"
score_thr   = 0.30    # 점수 필터 (과검출이면 ↑, 누락 많으면 ↓)
iou_thr     = 0.50    # 같은 약의 겹친 앵커를 하나로 묶는 IoU 기준
visualize_n = 8       # 앞 N장만 시각화 (0이면 비활성)

model.eval()

# label index(1..N) -> 실제 카테고리 id 매핑 (Dataset에서 만든 맵 사용)
idx_to_catid = getattr(full_ds, "idx_to_catid", None)
if idx_to_catid is None:
    idx_to_catid = {i+1: cid for i, cid in enumerate(full_ds.cat_ids)}

rows = []
annotation_id = 1
shown = 0

file_list = sorted([fn for fn in os.listdir(test_dir)
                    if fn.lower().endswith((".png", ".jpg", ".jpeg"))])

for fn in file_list:
    img_path = os.path.join(test_dir, fn)
    image_id = os.path.splitext(fn)[0]
    img = Image.open(img_path).convert("RGB")
    x = T.ToTensor()(img).to(device)

    with torch.no_grad():
        out = model([x])[0]

    # 텐서 정리 (CPU/float32)
    boxes  = out["boxes"].detach().to("cpu", dtype=torch.float32)
    labels = out["labels"].detach().to("cpu", dtype=torch.int64)
    scores = out["scores"].detach().to("cpu", dtype=torch.float32)

    # 1) score/bg 필터 (배경 label==0 제거)
    keep = (scores >= score_thr) & (labels != 0)
    boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

    print(boxes, labels, scores)
    if boxes.numel() == 0:
        continue

    kept_b, kept_l, kept_s = [], [], []

    # 2) 약(=category_id)별로 NMS → 3) 남은 것 중 '면적 최대' 1개만 유지
    for lab in labels.unique().tolist():
        m = (labels == lab)
        b = boxes[m]     # [M,4]
        s = scores[m]    # [M]

        if b.numel() == 0:
            continue

        cat_id = int(idx_to_catid.get(int(lab), int(lab)))
        print(f"\n🔹 Class {cat_id} — NMS 전: {len(b)}개")
        for i in range(len(b)):
            x1, y1, x2, y2 = b[i].tolist()
            area = (x2 - x1) * (y2 - y1)
            print(f"  ├─ idx={cat_id} score={s[i]:.3f} area={area:.1f} box=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})")

        # NMS로 겹친 앵커 정리 (동일 class 내)
        keep_idx = ops.nms(b, s, iou_thr)
        b = b[keep_idx]
        s = s[keep_idx]

        # 같은 약(class)에서도 여러 위치가 남을 수 있으므로 '면적 가장 큰' 1개만 선택
        if b.shape[0] > 1:
            areas = (b[:,2]-b[:,0]).clamp(min=0) * (b[:,3]-b[:,1]).clamp(min=0)
            best = torch.argmax(areas)
            b = b[best:best+1]
            s = s[best:best+1]

        kept_b.append(b)
        kept_l.append(torch.full((b.shape[0],), lab, dtype=torch.int64))
        kept_s.append(s)

    if not kept_b:
        continue

    boxes_kept  = torch.cat(kept_b, dim=0)
    labels_kept = torch.cat(kept_l, dim=0)
    scores_kept = torch.cat(kept_s, dim=0)

    # 4) 시각화
    if shown < visualize_n:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        ax.imshow(img); ax.axis("off")

    for (x1, y1, x2, y2), lab, sc in zip(boxes_kept.tolist(),
                                         labels_kept.tolist(),
                                         scores_kept.tolist()):
        cat_id = int(idx_to_catid.get(int(lab), int(lab)))
        rows.append({
            "annotation_id": annotation_id,
            "image_id": image_id,                       # 확장자 없는 파일명
            "category_id": cat_id,                      # 실제 카테고리 id
            "bbox_x": round(float(x1), 2),
            "bbox_y": round(float(y1), 2),
            "bbox_w": round(float(x2 - x1), 2),
            "bbox_h": round(float(y2 - y1), 2),
            "score":  round(float(sc), 3),
        })
        annotation_id += 1

        if shown < visualize_n:
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1,
                                     linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
            ax.text(x1, max(0, y1-5), f"{cat_id} ({sc:.2f})",
                    fontsize=9, color="white",
                    bbox=dict(facecolor="black", alpha=0.6, pad=1))

    if shown < visualize_n:
        shown += 1
        ax.set_title(f"{fn} — per-drug NMS → largest ONE per class (kept {boxes_kept.shape[0]})", fontsize=12)
        plt.show()

# 5) CSV 저장
df = pd.DataFrame(rows, columns=[
    "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
])
df.to_csv(csv_out, index=False, encoding="utf-8-sig")
print(f"✅ CSV 저장 완료(약별 가장 큰 1개만): {csv_out} (rows={len(df)})")

Output hidden; open in https://colab.research.google.com to view.